In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv')
df.head(10)

In [ ]:
y = df["User Rating"].value_counts()
y.plot(kind="pie")

In [ ]:
fig = px.histogram(df, x="User Rating",labels={'':'The Number of Books'},title="User Rating Histogram")
fig.show()

### Conclusion:
* About 180 books have a rating of 4.8 or 4.9.
* In the meantime, the rating of 9 books is less than 4.0.

In [ ]:
authors = df["Author"].value_counts()
author_times = []
author_names = []
for i in range(len(authors)):
    author_names.append(authors.keys()[i])
    author_times.append(authors[i])

In [ ]:
popular_authors = pd.DataFrame({"Author":author_names,"Number of Times":author_times})
popular_authors

In [ ]:
df[['User Rating','Author','Name']].groupby(by=['Author','Name']).mean().sort_values(by
=['User Rating'],ascending=False).head(10)

In [ ]:
df[['Name','User Rating','Year']].groupby(by='Name').mean().sort_values(by=
'User Rating',ascending=False).head(5)

In [ ]:
df['Name_len']=df['Name'].str.len()

In [ ]:
px.scatter(df.sort_values(by='Year'),x='User Rating',y='Price', animation_frame='Year', color='Genre',size='Name_len', 
title="Over all book Rating & Price by every year")

In [ ]:
!pip install sweetviz

In [ ]:
import sweetviz as sv
sweet_report = sv.analyze(df)
sweet_report.show_html('sweet_report.html')

#### Executing SweetViz will save a html file of the Automated visualizations, which you can download from the Kaggle Output files!

In [ ]:
df.Price.plot(kind = 'hist',bins = 50,figsize = (10,4))
plt.show()

In [ ]:
year = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
num_fic= []
for i in year:
    fic = df[(df["Year"] == i) & (df["Genre"] == "Fiction")]
    num_fic.append(len(fic))
df_fict = pd.DataFrame({"Year": year,"numberofFiction": num_fic})
df_fict.head()

In [ ]:
year = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
num_nonfic= []
for i in year:
    nonfic = df[(df["Year"] == i) & (df["Genre"] == "Non Fiction")]
    num_nonfic.append(len(nonfic))
df_nonfict = pd.DataFrame({"Year": year,"numberofNonFiction": num_nonfic})
df_nonfict.head()

In [ ]:
df_nonfict["numberofFiction"] = df_fict["numberofFiction"]
df_nonfict.head()

In [ ]:
df_genre = df_nonfict.copy()
df_genre.head()

In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
plt.bar(x = df_genre["Year"],height=df_genre["numberofNonFiction"],)
plt.plot(df_genre["Year"],df_genre["numberofNonFiction"],"go-")
#plt.plot(df_genre["Year"],df_genre["numberofFiction"],"bo-")
plt.title("Trend of having NonFiction on BestSeller Books List(2009-2019)")
plt.xlabel("Year")
plt.ylabel("No. of NonFiction published")

plt.subplot(1,2,2)
plt.bar(x = df_genre["Year"],height=df_genre["numberofFiction"],)
plt.plot(df_genre["Year"],df_genre["numberofFiction"],"bo-")
plt.title("Trend of Having Fiction on BestSeller Books List(2009-2019)")
plt.xlabel("Year")
plt.ylabel("No. of Fiction published")
plt.show()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
#statistics
df.describe()

In [ ]:
#no of rows and columns
df.shape

In [ ]:
df.head()

In [ ]:
#check correlation and plot heatmap 
correlation = df.corr()
plt.figure(figsize = (10,6))
sns.heatmap(correlation,annot=True)
plt.show()

In [ ]:
df['Name'].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder

df = df.copy(deep = True)
LE = LabelEncoder()
df['Genre'] = LE.fit_transform(df['Genre'])

In [ ]:
df['Author'] = LE.fit_transform(df['Author'])
df['Name'] = LE.fit_transform(df['Name'])

In [ ]:
df.head()

In [ ]:
X=df.drop(['User Rating'],axis=1)
y=df[['User Rating']]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=8)

In [ ]:
from catboost import CatBoostRegressor
CB = CatBoostRegressor(iterations=603,
                          learning_rate=0.35,
                          depth=8)
CB.fit(X_train,y_train)

In [ ]:
y_pred=CB.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse